In [1]:
import dgl
import dgl.function as fn
import torch


from iou_graph import IOUGraph
from dgl_reflacx_tools.tools import gridify, grid_readout

from dgl_reflacx_tools.dgl_reflacx_collection import GraphCollection

/home/antonio/dev/pyvenvs/graph_classification/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_pth = 'datasets/reflacx_densnet225_iou'
collection = GraphCollection(dataset_pth, IOUGraph)

loading metadata
metadata loaded from file
Done loading data from cached files.


#### Getting sample batch

In [12]:
batch_size = 5
grid_size = 4

In [9]:
pairs = [collection.fetch_by_dgl_index(i) for i in range(batch_size)]

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 3 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 4 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 5 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 18 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 20 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 28 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 30 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 31 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 53 out of chest bounding box

34cedb74-d0996b40-6d218312-a9174bea-d48dc033 -- P102R108387 
  Fixation 55 out of che

In [35]:
graphs = [pair.dgl_graph for pair in pairs]
labels = [pair.dgl_labels for pair in pairs]


In [36]:
batch = dgl.batch(graphs)
grid = gridify(batch, grid_size)
labels = torch.cat(labels).reshape((batch_size, len(labels[0])))

batch, (len(grid), len(grid[0])), labels.shape

(Graph(num_nodes=721, num_edges=12067,
       ndata_schemes={'feats': Scheme(shape=(1024,), dtype=torch.float32), 'duration': Scheme(shape=(), dtype=torch.float32), 'norm_bottom_right': Scheme(shape=(2,), dtype=torch.float32), 'norm_top_left': Scheme(shape=(2,), dtype=torch.float32), 'norm_y': Scheme(shape=(), dtype=torch.float32), 'norm_x': Scheme(shape=(), dtype=torch.float32)}
       edata_schemes={'weight': Scheme(shape=(), dtype=torch.float32)}),
 (4, 4),
 torch.Size([5, 6]))

#### initialization

In [38]:
batch.ndata['conv_feats'] = batch.ndata['feats'].clone()

In [43]:
batch.ndata.keys()

dict_keys(['feats', 'duration', 'norm_bottom_right', 'norm_top_left', 'norm_y', 'norm_x', 'conv_feats'])

In [42]:
grid[0][0].ndata.keys()

dict_keys(['feats', 'duration', 'norm_bottom_right', 'norm_top_left', 'norm_y', 'norm_x', '_ID'])

In [46]:
grid[0][0].apply_edges(fn.u_mul_e('feats', 'weight', 'w_feats'))
grid[0][0].update_all(fn.e_div_v('w_feats', 'duration', 'norm_feats'),
                fn.sum('norm_feats', 'blabla'))

In [47]:
grid[0][0].ndata.keys()

dict_keys(['feats', 'duration', 'norm_bottom_right', 'norm_top_left', 'norm_y', 'norm_x', '_ID', 'blabla'])

In [49]:
batch.ndata.keys()

dict_keys(['feats', 'duration', 'norm_bottom_right', 'norm_top_left', 'norm_y', 'norm_x', 'conv_feats'])